# Exploratory Data Analysis - Valorant

## Libs

### Data manipulation

In [2]:
import pandas as pd
from datetime import datetime
import pathlib

### Charts

In [3]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

## Initial Settings and Loads

### Pathlib

In [4]:
abs_path = pathlib.Path.cwd().parent

### Load Players Rank

In [5]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('players').joinpath('players_processed.csv')
df_players_raw = pd.read_csv(file_read)
del df_players_raw['Unnamed: 0']

### Load Matches

In [6]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('matches').joinpath('matches_processed.csv')
df_matches_raw = pd.read_csv(file_read)
del df_matches_raw['Unnamed: 0']

### Load Weapons

In [7]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('weapons').joinpath('weapons_processed.csv')
df_weapons_raw = pd.read_csv(file_read)
del df_weapons_raw['Unnamed: 0']

## EDA Players

### Selected Players

In [8]:
selected_players_list = (
    df_players_raw
    .sort_values('rankedRating', ascending = False)
    .reset_index(drop = True)
    .head(10)
    )
selected_players_list = selected_players_list['userId'].to_list()
selected_players_list

['neveR%23god',
 'RieNs%23wagwn',
 'SEN zekken%235193',
 'SR mada%231945',
 'TBK anao sato%23lukxo',
 'artzin%23psz',
 'Heretics AvovA%23uwu',
 'TBG lurzyyyy%23Boss',
 'TSM NaturE%23RAT',
 'KC TakaS%23KLWM']

O rankedRating é uma média da pontuação entre derrotas e vitórias que um jogador tem, ou seja, quando um jogador ganha uma partida ele obtem um quantia de pontos determinada a partir de sua performance na partida e quando ele perde uma partida ele sofre um decréscimo de pontos de acordo com sua performance na partida. Na lista acima estamos selecionando os 10(dez) jogadores com maior pontuação ranqueada de todos os servidores (BR, NA, EA, JP, AP, KR, LATAM) do jogo em questão.

### Ranking

In [9]:
players_ranking = (
    df_players_raw[['userId', 'server', 'leaderboardRank', 'rankedRating', 'avgScore', 'competitiveTier']]
    .sort_values('rankedRating', ascending = False)
    .reset_index(drop = True)
    .head(10)
    )

In [10]:
fig = px.scatter(players_ranking, x='userId', y='rankedRating')
fig.show()

In [11]:
fig = px.scatter(players_ranking, x='userId', y='avgScore')
fig.show()

In [12]:
players_ranking.sort_values(['leaderboardRank'], ascending = True)

,userId,server,leaderboardRank,rankedRating,avgScore,competitiveTier
0,neveR%23god,eu,1,1252,275.0,27
2,SEN zekken%235193,na,1,1159,268.0,27
4,TBK anao sato%23lukxo,br,1,1127,249.0,27
1,RieNs%23wagwn,eu,2,1183,266.0,27
3,SR mada%231945,na,2,1147,249.0,27
5,artzin%23psz,br,2,1122,261.0,27
7,TBG lurzyyyy%23Boss,eu,3,1119,245.0,27
8,TSM NaturE%23RAT,na,3,1119,266.0,27
6,Heretics AvovA%23uwu,eu,4,1119,251.0,27
9,KC TakaS%23KLWM,eu,5,1107,268.0,27


No ranqueamento dos jogadores podemos observar suas posições nos ranques de seus determinados servidores, como por exemplo o jogador neveR%23god ser o primeiro do ranque europeu assim como o artzin%23psz é o primeiro jogador do ranque brasileiro. Mas como precisamos distinguir os jogadores utilizamos os campos rankedRating e avgScore para simularmos um ranque global, sendo assim o jogador neveR%23god se torna o top 1 do nosso ranque SEN zekken%235193 top 2 TBK anao sato%23lukxo top 3 e assim sucessivamente.

Observando os gráficos a cima, podemos ver que há uma correlação entre os campos rankedRating e avgScore, nos evidenciando que para ter um alto rankedRating é necessário ter um avgScore a cima do 240,  outro ponto que podemos perceber que é insustentavel manter uma avgScore acima dos 300 pontos, pois nem os melhores jogadores do mundo mantem uma média dessas. Assim nos dando margem até para tentarmos previnir hackers utilizando esses parametros como base.

### Matches

In [13]:
matches_per_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby(['userId'])['matchId']
                        .count()
                        .reset_index(name = 'count')
                        .reset_index(drop = True)
                        )

In [14]:
fig = px.scatter(matches_per_player, x = 'userId', y = 'count', title = 'Number of matches per player')
fig.show()

Olhando para os dados a cima vimos que todos os 10 jogadores possuem a mesma quantidade de partidas neste dataset. Isso ocorre pois nosso crawler lê uma pagina que trás apenas os dados das ultimas 18 partidas e em 10 paginas, sendo assim (18 * 10 = 180 partidas). Logo sabemos que nossos dados e analises abaixo não seram afetados por diferenças entre as quantidades de partidas jogados entre os jogadores selecionados.

### Time

In [15]:
total_time_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['playtimeHoursValue']]
    .sum()
    .reset_index()
)


In [16]:
fig = px.scatter(total_time_player, x = 'userId', y = 'playtimeHoursValue', color = 'playtimeHoursValue', title = 'Total de horas por jogador')
fig.show()

In [17]:
fig = px.box(total_time_player, y="playtimeHoursValue", title = 'Intervalo entre quantidade de horas jogadas')
fig.show()

No tocante a quantidade de horas jogadas, podemos analisar que não tem uma correlação direta entre com a performance do jogadore seja em rank ou pontuação. Podemos pegar como exemplo o jogador SR mada%231945 do servidor(na) ele é o jogador com a menor quantidade de horas jogadas(96.35 horas) e consegue performar mais que o nosso jogador mais aciduo em quantidade de horas no caso o artzin%23psz com o total de horas de (105.01 horas).

### Results

In [18]:
results_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby(['userId', 'result'])['userId']
                        .count()
                        .reset_index(name = 'count')
                        .sort_values(['count'], ascending = False)
                        .reset_index(drop = True)
                        )

In [19]:
fig = px.bar(results_player, x = 'userId', y = 'count', barmode='group', color = 'result', title = 'Resultados por jogador')
fig.show();

In [20]:
fig = px.box(results_player, x = "result", y = "count", color = "result")
fig.update_traces(quartilemethod="exclusive")
fig.show()

Com base nas analíses a cima podemos avaliar que os resultados não estão diretamente relacionados ao posicionamento dos jogadores seja por performance (rankedRating ou avgScore) ou ordem na lista dos ranqueados. Dito isso sabemos que o jogador que mais perde partidas (TBK anao sato%23lukxo) pode ter uma performance individual superior a aquele que menos perde (KC TakaS%23KLWM).

Vale ressaltar que a vitória de uma partida esta relacionada a colaboração do time do que só performance individual. Em minhas partidas sinto muito mais facilidade quando o time joga bem onde todos exercem suas funções do que quando tem um unico player jogando acima da média.

### Headshots

In [21]:
number_headshot_per_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['headshotsValue']
    .sum()
    .reset_index(name = 'sum')
    .sort_values('sum', ascending=False)
    .reset_index(drop = True)
)

In [22]:
fig = px.scatter(number_headshot_per_player, x = 'userId', y = 'sum', title = 'Quantidade de headshot por jogador')
fig.show()

In [23]:
fig = px.box(number_headshot_per_player, y="sum", title = 'Headshot')
fig.show()

Os jogadores e publicos mais intusiastas sabem que o headshot(HS) é um ponto auto em uma partida,acertar um HS mostra suas habilidades em mira e até mesmo memoria muscular.
Mas como podemos ver nos graficos a cima o headshot não acrescenta nada ao resultado final de uma partida.

Com 3.104 HS o jogador SEN zekken%235193 se mostra um eximio atirador se tornando um outlier com um percentual de acertos na casa dos 80% em comparação a média de acertos dos outros competidores selecionados. Vale ter em mente que o SEN zekken%235193 é o primeiro colocado no servidor norte americano(NA) e nosso terceiro colocado no rank global.

### KDA

In [56]:
number_kda_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby('userId')[['killsValue', 'deathsValue', 'assistsValue',]]
                        .sum()
                        .sort_values(['killsValue'], ascending = False)
                        .reset_index()
                        )

In [64]:
fig = go.Figure()

fig.add_trace(go.Bar(x = number_kda_player['userId'], y = number_kda_player['killsValue'], name = 'killsValue', text = number_kda_player['killsValue']))
fig.add_trace(go.Bar(x = number_kda_player['userId'], y = number_kda_player['deathsValue'], name = 'deathsValue', text = number_kda_player['deathsValue']))
fig.add_trace(go.Bar(x = number_kda_player['userId'], y = number_kda_player['assistsValue'], name = 'assistsValue', text = number_kda_player['assistsValue']))

fig.show()

In [51]:
number_kda_player = number_kda_player.drop(columns = 'userId')

fig = go.Figure()

for col in number_kda_player:
  fig.add_trace(go.Box(y=number_kda_player[col].values, name=number_kda_player[col].name))
  
fig.show()

O KDA (Kill, Deaths and Assists) é um bom termometro para indicar se um jogador foi bem ou não em uma partida. Em nossa seleção de jogadores o artzin%23psz (K = 3.732 D = 2.759 A = 825) é o que mais se destaca no tocante a Kills tendo o numero 3.732 kills em seu portifolio. Porém se comparmos o artzin%23psz com outros jogadores, por exemplo o Heretics AvovA%23uwu(K = 3.385 D = 2.472 A = 1.039), podemos ver que mesmo morrendo menos e dando mais assistencia a metrica de kills ainda é relevante mas não o suficiente para posicionar o jogador com mais numeros de kills ou melhores numeros em assistencias ou morte em boas posições do nosso rank global. Mas vale ressaltar que o Heretics AvovA%23uw se torna nosso outlier no quesito a players que menos morreram.

### Damage

In [73]:
number_damage_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['damageValue','damageReceivedValue']
    .sum()
    .reset_index()
    .sort_values('damageValue', ascending = False)
)

In [77]:
fig = go.Figure()
fig.add_trace(go.Bar(x=number_damage_player['userId'], y=number_damage_player['damageValue'], text = number_damage_player['damageValue'],
                    name='damageValue'))
fig.add_trace(go.Bar(x=number_damage_player['userId'], y=number_damage_player['damageReceivedValue'], text = number_damage_player['damageReceivedValue'],
                    name='damageReceivedValue'))
fig.show()

In [78]:
number_damage_player = number_damage_player.drop(columns = 'userId')

fig = go.Figure()

for col in number_damage_player:
  fig.add_trace(go.Box(y=number_damage_player[col].values, name=number_damage_player[col].name))
  
fig.show()

Nossa métrica de dano é feita da soma do dano causado por jogador, por exemplo quando um jogador elimina um adiversario e da qualquer outro tipo de dano, seja com projeteis de sua arma ou habilidade. Como podemos constatar também não impacta diretamente nosso rank global. Mas um ponto interessante é que dos nossos jogadores selecionados o dano recebido nunca é maior que o dano acertado, logo podemos dizer que nossos players matam mais que morrem e possuem certa pericia para não serem alvejados por projeteis e habilidades.

### Economy

In [82]:
number_economy_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['econRatingValue']
    .mean()
    .sort_values(ascending = False)
    .reset_index()
)

In [93]:
fig = go.Figure()
fig = px.bar(x=number_economy_player['userId'], y=number_economy_player['econRatingValue'], color=number_economy_player['econRatingValue'], text= number_economy_player['econRatingValue'].round(3))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=number_economy_player['econRatingValue'], name = 'econRatingValue' ))
fig.show()

Podemos analisar que a metrica de economia é a primeira caracterisca que esta relacionada as posições do nosso rank global. Como podemos ver o jogador neveR%23god é o player com o maior econRating tendo a marca de 72.933.

A métrica de economia dentro do jogo nos diz o quão eficiente foi os nossos gastos, basicamente nos o jogo leva em consideração a quantia gasta por um jogador em uma partida e o que ele conseguiu fazer, por exemplo se o player gastou 900 moedas e eliminou 3 adversarios ele com certeza tera o econRating muito maior do que o jogador que gastou 900 moedas e não fez nada significativo.

### Agents

In [102]:
number_matches_agents = (
                             df_matches_raw
                            .query('userId == @selected_players_list')
                            .groupby(['agentName'])['agentName']
                            .count() 
                            .reset_index(name='count') 
                            .sort_values(['count'], ascending=False) 
                            .reset_index(drop = True)
)

In [101]:
fig = px.bar(number_matches_agents, y='count', x='agentName', text_auto='.4s', orientation='v',
            title="Number of matches an agent has been selected", color=number_matches_agents['count'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)

fig.show()

In [162]:
number_matches_agents = (
                             df_matches_raw
                            .query('userId == @selected_players_list')
                            .groupby(['userId','agentName'])['agentName']
                            .count() 
                            .reset_index(name='count') 
                            .sort_values(['userId'], ascending=False) 
                            # .sort_values(['count'], ascending=False) 
                            .reset_index(drop = True)
                            
)

number_matches_agents

,userId,agentName,count
0,neveR%23god,Viper,3
1,neveR%23god,Reyna,3
2,neveR%23god,Raze,1
3,neveR%23god,Phoenix,63
4,neveR%23god,Jett,8
...,...,...,...
145,Heretics AvovA%23uwu,Phoenix,3
146,Heretics AvovA%23uwu,Raze,6
147,Heretics AvovA%23uwu,Sage,2
148,Heretics AvovA%23uwu,Viper,53


In [ ]:
players_ranking

,userId,server,leaderboardRank,rankedRating,avgScore,competitiveTier
0,neveR%23god,eu,1,1252,275.0,27
1,RieNs%23wagwn,eu,2,1183,266.0,27
2,SEN zekken%235193,na,1,1159,268.0,27
3,SR mada%231945,na,2,1147,249.0,27
4,TBK anao sato%23lukxo,br,1,1127,249.0,27
5,artzin%23psz,br,2,1122,261.0,27
6,Heretics AvovA%23uwu,eu,4,1119,251.0,27
7,TBG lurzyyyy%23Boss,eu,3,1119,245.0,27
8,TSM NaturE%23RAT,na,3,1119,266.0,27
9,KC TakaS%23KLWM,eu,5,1107,268.0,27


In [ ]:
number_matches_agents = (
                             df_matches_raw
                            .query('userId == @selected_players_list')
                            .groupby(['agentName'])['agentName']
                            .count() 
                            .reset_index(name='count') 
                            .sort_values(['count'], ascending=False) 
                            .reset_index(drop = True)
)

In [ ]:
number_matches_agents = (
                             df_matches_raw
                            .query('userId == @selected_players_list')
                            .groupby(['userId'])['agentName']
                            .max() 
                            .reset_index(name='count') 
                            .sort_values(['count'], ascending=False) 
                            .reset_index(drop = True)
)

In [ ]:
number_matches_agents

,userId,count
0,RieNs%23wagwn,Yoru
1,SR mada%231945,Yoru
2,TSM NaturE%23RAT,Yoru
3,artzin%23psz,Yoru
4,Heretics AvovA%23uwu,Viper
5,KC TakaS%23KLWM,Viper
6,SEN zekken%235193,Viper
7,TBG lurzyyyy%23Boss,Viper
8,TBK anao sato%23lukxo,Viper
9,neveR%23god,Viper


In [ ]:
fig = px.bar(number_matches_agents, y='count', x='userId', text_auto='.4s', orientation='v',
            title="Number of matches an agent has been selected")
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)

fig.show()

### Weapons

In [ ]:
# from IPython.display import display
# with pd.option_context('display.max_rows', 180, 'display.max_columns', 25):
#     display(df_weapons_raw.query('userId == @selected_players_list').reset_index(drop = True)).max_rows()

In [ ]:
df_weapons_raw.head(5)

,userId,weaponName,matchesPlayedValue,matchesWonValue,matchesLostValue,matchesTiedValue,matchesWinPctValue,roundsPlayedValue,killsValue,killsPerRoundValue,...,damageValue,damagePerRoundValue,damagePerMatchValue,damageReceivedValue,dealtHeadshotsValue,dealtBodyshotsValue,dealtLegshotsValue,killDistanceValue,avgKillDistanceValue,longestKillDistanceValue
0,TBK anao sato%23lukxo,marshal,565,289,259,17,51.150442,234,780,0.628205,...,128466,92.611111,227.373451,0,439,412,3,413923,732.607080,6837
1,TBK anao sato%23lukxo,classic,1195,608,561,26,50.878661,2012,1192,0.133698,...,137827,15.215209,115.336402,0,1244,1504,81,346357,289.838494,4575
2,TBK anao sato%23lukxo,phantom,1276,609,637,30,47.727273,916,5017,0.899563,...,805722,148.398472,631.443574,0,4221,5949,392,1483714,1162.785266,5426
3,TBK anao sato%23lukxo,shorty,122,70,50,2,57.377049,40,93,0.625000,...,10300,65.275000,84.426230,0,118,604,47,12410,101.721311,735
4,TBK anao sato%23lukxo,ares,40,27,13,0,67.500000,2,91,1.000000,...,11427,81.000000,285.675000,0,62,224,45,5082,127.050000,2639


#### Most used weapons

In [ ]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby('userId')['weaponName', 'roundsPlayedValue','matchesPlayedValue']
    .max()
    .reset_index()
    .sort_values('roundsPlayedValue', ascending = False)

)
df_weapons_most_used

,userId,weaponName,roundsPlayedValue,matchesPlayedValue
5,TBG lurzyyyy%23Boss,vandal,15919,5561
9,neveR%23god,vandal,8733,2582
3,SEN zekken%235193,vandal,8632,2132
4,SR mada%231945,vandal,8382,2575
2,RieNs%23wagwn,vandal,8215,3845
6,TBK anao sato%23lukxo,vandal,7458,2426
8,artzin%23psz,vandal,6773,3261
0,Heretics AvovA%23uwu,vandal,6749,1773
7,TSM NaturE%23RAT,vandal,5253,2105
1,KC TakaS%23KLWM,vandal,4487,1075


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['matchesPlayedValue'],
                    name='matchesPlayedValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['roundsPlayedValue'],
                    name='roundsPlayedValue', text='Vandal'))
fig.update_layout(title = 'Most used weapon')

fig.show()

#### Results by weapons

In [ ]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['weaponName','matchesWonValue', 'matchesLostValue', 'matchesTiedValue']]
    .max()
    .sort_values('matchesWonValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,weaponName,matchesWonValue,matchesLostValue,matchesTiedValue
0,TBG lurzyyyy%23Boss,vandal,2881,2611,69
1,RieNs%23wagwn,vandal,2041,1750,54
2,artzin%23psz,vandal,1697,1494,70
3,neveR%23god,vandal,1440,1104,40
4,SR mada%231945,vandal,1439,1108,28
5,SEN zekken%235193,vandal,1286,831,15
6,TBK anao sato%23lukxo,vandal,1214,1166,46
7,TSM NaturE%23RAT,vandal,1132,950,23
8,Heretics AvovA%23uwu,vandal,1038,709,26
9,KC TakaS%23KLWM,vandal,657,402,16


In [ ]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['killsValue', 'killsPerRoundValue']]
    .sum()
    .sort_values('killsValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,killsValue,killsPerRoundValue
0,TBG lurzyyyy%23Boss,100033,11.364133
1,RieNs%23wagwn,70020,12.880045
2,artzin%23psz,57882,73.910453
3,neveR%23god,49058,51.154913
4,SR mada%231945,47983,11.789442
5,TBK anao sato%23lukxo,44437,12.808882
6,SEN zekken%235193,43261,13.230447
7,TSM NaturE%23RAT,40182,12.333626
8,Heretics AvovA%23uwu,39157,20.402591
9,KC TakaS%23KLWM,20901,28.669701


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['killsValue'],
                    name='killsValue', text='Vandal'))

fig.update_layout(title = 'Kills by weapons')

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['killsPerRoundValue'],
                    name='killsPerRoundValue', text='Vandal'))
fig.update_layout(title = 'Kills by weapons')
fig.show()

#### Shoot stats by weapons

In [ ]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['dealtHeadshotsValue', 'dealtBodyshotsValue', 'dealtLegshotsValue']]
    .sum()
    .sort_values('dealtHeadshotsValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,dealtHeadshotsValue,dealtBodyshotsValue,dealtLegshotsValue
0,TBG lurzyyyy%23Boss,82271,90386,6090
1,RieNs%23wagwn,51061,82895,5312
2,SR mada%231945,37940,52411,4238
3,artzin%23psz,37739,81351,5447
4,TBK anao sato%23lukxo,35261,42922,2354
5,SEN zekken%235193,33006,51105,4093
6,Heretics AvovA%23uwu,30464,46786,3354
7,neveR%23god,30114,55290,2846
8,TSM NaturE%23RAT,28954,51940,3413
9,KC TakaS%23KLWM,13915,26794,1629


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtHeadshotsValue'],
                    name='dealtHeadshotsValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtBodyshotsValue'],
                    name='dealtBodyshotsValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtLegshotsValue'],
                    name='dealtLegshotsValue', text='Vandal'))
fig.update_layout(title = 'Shoot stats by weapons')
fig.show()

## EDA Matches

### Days

In [ ]:
matches_per_day = (
    
                    df_matches_raw
                    .query("userId == @selected_players_list")
                    .groupby([df_matches_raw.dateMatch, df_matches_raw.weekDay])['matchId']
                    .count()
                    .reset_index(name='count')
                    .reset_index(drop = True)
                    )

In [ ]:
fig = px.line(x = matches_per_day['dateMatch'], y = matches_per_day['count'], title= 'Number of matches per day')
fig.show()

In [ ]:
fig = px.line(x = matches_per_day['dateMatch'], y = matches_per_day['count'], title= 'Number of matches per day')
fig.show()

In [ ]:
weekDayOrdered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
mean_matches_per_days_week = (
                                matches_per_day.groupby(matches_per_day.weekDay)['count']
                                .mean()
                                .reindex(weekDayOrdered) 
                                .reset_index(name = 'mean')
                                .reset_index(drop = True)
                                .round(3)
                                )

In [ ]:
fig = px.line(mean_matches_per_days_week, x = 'weekDay', y = 'mean', title = 'Means of matches per days of week')
fig.show()

In [ ]:
means_time_matches = df_matches_raw[['matchId', 'weekDay', 'playtimeValue']]
means_time_matches['playtimeValueMinutes'] = means_time_matches.playtimeValue/60

means_time_matches = (
                        means_time_matches.groupby('weekDay')['playtimeValueMinutes']
                        .mean()
                        .reindex(weekDayOrdered)
                        .reset_index(name = 'mean')
                        .reset_index(drop = True)                     
                        .round(2)
)

In [ ]:
fig = px.line(means_time_matches, x = 'weekDay', y = 'mean', title = 'Means time per days of week')
fig.show()

In [ ]:
top_seven_days_matches = (
                            df_matches_raw.groupby(['dateMatch', 'weekDay'])['weekDay']
                            .count()
                            .reset_index(name = 'count')
                            .reset_index(drop = True)
                            .sort_values('count', ascending = False)
                            .head(7)
                            )

In [ ]:
fig = px.bar(top_seven_days_matches, x='weekDay', y='count',
             hover_data=['dateMatch', 'count'], color='count', height=400, title = 'Top seven days with most matches played')
fig.show()

### Plants and Defuses

In [ ]:
matches_plat_defuse = (
                        df_matches_raw.query('userId == @selected_players_list').groupby(['result'])[['plantsValue', 'defusesValue']]
                        .sum()
                        .sort_values('plantsValue', ascending = False)
                        .reset_index()
)

In [ ]:
fig = px.bar(matches_plat_defuse, x = 'result', y = 'plantsValue', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

fig = px.bar(matches_plat_defuse, x = 'result', y = 'defusesValue', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=matches_plat_defuse['plantsValue'],
                    name='plantsValue'))
fig.add_trace(go.Box(y=matches_plat_defuse['defusesValue'],
                    name='defusesValue'))
fig.show()